In [1]:
%pip install transformers SentencePiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00


In [2]:
!pip install "git+https://github.com/evalplus/evalplus.git" --upgrade

  Cloning https://github.com/evalplus/evalplus.git to /tmp/pip-req-build-vdgfjuw8
  Running command git clone --filter=blob:none --quiet https://github.com/evalplus/evalplus.git /tmp/pip-req-build-vdgfjuw8
  Resolved https://github.com/evalplus/evalplus.git to commit 2930c05c6c4f1c2e6ffdf03d811aaff3fbf32df8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for evalplus: filename=evalplus-0.1.6-py3-none-any.whl size=34310 sha256=7a06fa34825d7379ab4b1f3ce0581c9982f9da32952d96cd547d5fd9449e1c92
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwjpfp67/wheels/8d/1b/61/998e41af57d23b39fd3bb4ba63d9439900cb02a192233df4b3
  Created wheel for tempdir: filename=tempdir-0.7.1-py3-none-any.whl size=2196 sha256=bd5e47c15cccd55f6bed816068697e9d412384bd7ad88ed75a30b886ee79c17a


In [3]:
import os
import re
from tqdm import tqdm
import transformers, torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [ ]:
start_index = 0
end_index = 164
max_len = 600
STOP_SEQS = ['\nclass', '\ndef', '\n#', '\nif', '\nprint']
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
## problems in evalplus
from evalplus.data import get_human_eval_plus, write_jsonl

plus_problems = get_human_eval_plus()

plus_task_ids = sorted(plus_problems.keys())[start_index: end_index]
plus_prompts = [plus_problems[task_id]['prompt'] for task_id in plus_task_ids]
num_samples = len(plus_prompts)
print("Number of samples: {}".format(num_samples))

Number of samples: 164


In [ ]:
def generate_completion_samples_codeT5(model,temp, output_file,loop):
  ## defining model
  model = model
  output_file = output_file

  tokenizer = AutoTokenizer.from_pretrained(model)

  model = AutoModelForSeq2SeqLM.from_pretrained(model,
                                                trust_remote_code=True,
                                                torch_dtype=torch.float16,
                                                low_cpu_mem_usage=True)

  model.eval()
  model.to(DEVICE)
  completion_seqs = []
  loops = loop

  for i in tqdm(range(num_samples), ncols=0, total=num_samples):
    prompt = plus_prompts[i].replace('    ', '\t')

    prompt_batch_decoder = [prompt]
    ids_batch = [plus_task_ids[i]]

    encoding_decoder = tokenizer(prompt_batch_decoder, return_tensors="pt", truncation=True, max_length=max_len).to(DEVICE)

    for _ in tqdm(range(loops), total=loops, leave=False, ncols=0):

      with torch.no_grad():
        gen_tokens = model.generate(**encoding_decoder,
                                  decoder_input_ids=encoding_decoder['input_ids'],
                                  do_sample=True,
                                  temperature=temp,
                                  max_length=max_len,
                                  decoder_start_token_id=tokenizer.pad_token_id,
                                  eos_token_id=tokenizer.eos_token_id,
                                  top_p=0.95)
      gen_tokens = gen_tokens[:, encoding_decoder['input_ids'].shape[-1]:]

      gen_seqs = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)

      if gen_seqs is not None:
        assert len(ids_batch) == 1
        task_id = ids_batch[0]

        for seq_idx, gen_seq in enumerate(gen_seqs):

          completion_seq = gen_seq
          for stop_seq in STOP_SEQS:
              index = completion_seq.find(stop_seq)
              if index != -1:
                  completion_seq = completion_seq[:index]
          completion_seq = completion_seq.replace('\t', '    ')
          all_code = prompt.replace('\t', '    ') + completion_seq

          completion_seqs.append(
              {'task_id': task_id,
                'completion': completion_seq
                }
            )



  print("Saving results to {}".format(output_file))

  write_jsonl(output_file, completion_seqs)



In [ ]:
def generate_completion_samples_llama(model,temp, output_file,loop):
  ## defining model
  model = model
  output_file = output_file

  tokenizer = LlamaTokenizer.from_pretrained(model)

  model = LlamaForCausalLM.from_pretrained(model,
                                                trust_remote_code=True,
                                                torch_dtype=torch.float16,
                                                low_cpu_mem_usage=True)

  model.eval()
  model.to(DEVICE)
  completion_seqs = []
  loops = loop

  generation_config = transformers.GenerationConfig(
    do_sample=True,
    temperature=temp,
    top_p=0.95,
    max_new_tokens = max_len,
  )

  for i in tqdm(range(num_samples), ncols=0, total=num_samples):
    prompt = plus_prompts[i].replace('    ', '\t')

    prompt_batch_decoder = [prompt]
    ids_batch = [plus_task_ids[i]]

    encoding_decoder = tokenizer(prompt_batch_decoder, return_tensors="pt", truncation=True, max_length=max_len).to(DEVICE)
    input_ids=encoding_decoder['input_ids']

    for _ in tqdm(range(loops), total=loops, leave=False, ncols=0):

      with torch.no_grad():
        gen_tokens = model.generate(
                                  input_ids=input_ids,
                                  attention_mask=torch.ones_like(input_ids),
                                  generation_config=generation_config
                                  )

      gen_tokens = gen_tokens[:, encoding_decoder['input_ids'].shape[-1]:]

      gen_seqs = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)

      if gen_seqs is not None:
        assert len(ids_batch) == 1
        task_id = ids_batch[0]

        for seq_idx, gen_seq in enumerate(gen_seqs):

          completion_seq = gen_seq
          for stop_seq in STOP_SEQS:
              index = completion_seq.find(stop_seq)
              if index != -1:
                  completion_seq = completion_seq[:index]
          completion_seq = completion_seq.replace('\t', '    ')
          all_code = prompt.replace('\t', '    ') + completion_seq

          completion_seqs.append(
              {'task_id': task_id,
                'completion': completion_seq
                }
            )



  print("Saving results to {}".format(output_file))

  write_jsonl(output_file, completion_seqs)

In [ ]:
generate_completion_samples_codeT5("Salesforce/codet5p-2b",0.2,"codet5_2b_samples.jsonl",1)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

  0% 0/164 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:45<00:00, 45.45s/it]
  1% 1/164 [00:45<2:03:33, 45.48s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:40<00:00, 40.65s/it]
  1% 2/164 [01:26<1:55:09, 42.65s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:12<00:00, 12.16s/it]
  2% 3/164 [01:38<1:17:06, 28.73s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:40<00:00, 40.86s/it]
  2% 4/164 [02:19<1:29:24, 33.53s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:41<00:00, 41.11s/it]
  3% 5/164 [03:00<1:36:06, 36.27s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:15<00:00, 15.21s/it]
  4% 6/164 [03:15<1:16:39, 29.11s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:07<00:00,  7.16s/it]
  4% 7/164 [03:22<57:25, 21.94s/it]  
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:41<00:00, 41.20s/it]
  5% 8/164 [04:03<1:13:00, 28.08s/it]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:08<00:00,  8.89s/it]
  5% 9/164 [04:12<57:03, 22.09s/it]  
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:07<00:00,  7.43s/it]
  6% 10/164 [04:20<45:05, 17.5

Saving results to llama_7b_hf_samples.jsonl


In [ ]:
generate_completion_samples_llama("decapoda-research/llama-7b-hf",0.2,"llama_7b_hf_samples.jsonl",1)